# Introduction

This dataset was scraped from [nextspaceflight.com](https://nextspaceflight.com/launches/past/?page=1) and includes all the space missions since the beginning of Space Race between the USA and the Soviet Union in 1957!

### Install Package with Country Codes

In [ ]:
# !pip install iso3166

### Upgrade Plotly

Run the cell below if you are working with Google Colab.

In [ ]:
# %pip install --upgrade plotly

### Import Statements

In [ ]:
import numpy as np
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

# These might be helpful:
from iso3166 import countries_by_name
from datetime import datetime, timedelta
import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)
warnings.simplefilter(action="ignore", category=pd.errors.SettingWithCopyWarning)

### Notebook Presentation

In [ ]:
pd.options.display.float_format = '{:,.2f}'.format

### Load the Data

In [ ]:
df_data = pd.read_csv('mission_launches_updated.csv')

# Preliminary Data Exploration

* What is the shape of `df_data`? 
* How many rows and columns does it have?
* What are the column names?
* Are there any NaN values or duplicates?

In [ ]:
df_data

In [ ]:
df_data.isna().sum()

In [ ]:
df_data.columns

## Data Cleaning - Check for Missing Values and Duplicates

Consider removing columns containing junk data. 

In [ ]:
df_data.duplicated().sum()

In [ ]:
# merubah kolom datetime ke format date
dates = []
for date in df_data.Datetime: 
  split = date.split(" ")
  dt = f'{split[0]} {split[1]} {split[2]} {split[3]}'
  try:
    d = datetime.strptime(dt, "%a %b %d, %Y")
  except ValueError as e: 
    d = datetime.strptime(date, "%a %b %d, %Y")
  finally:
    dates.append(d)
df_data.Datetime = dates
df_data.dtypes

In [ ]:
df_data["Year"] = df_data["Datetime"].dt.year

In [ ]:
df_data['Price'] = pd.to_numeric(df_data['Price'], errors='coerce')

In [ ]:
#deal with NaN data
df_data['Price'] = df_data.groupby('Organisation')['Price'].transform(lambda x: x.fillna(x.mean()))
df_data['Price'] = df_data.groupby('Year')['Price'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
df_data.drop(columns=['Unnamed: 0'], inplace=True)

## Descriptive Statistics

In [ ]:
df_data.describe()

In [ ]:
df_data.info()

In [ ]:
df_data.isna().any()

# Number of Launches per Company

Create a chart that shows the number of space mission launches by organisation.

In [ ]:
organisation_launches=df_data['Organisation'].value_counts().sort_values(ascending=False)[:20]
organisation_launches

In [ ]:
fig = px.bar(
    x=organisation_launches.index, 
    y=organisation_launches.values, 
    title="Number of Launches per Organization",
    labels={"Organization": organisation_launches.index, "Launches": organisation_launches.values},
    color=organisation_launches.values,  # Menggunakan warna berdasarkan jumlah peluncuran
    text=organisation_launches.values  # Menambahkan nilai angka pada setiap bar
)
# Menambahkan rotasi pada label sumbu x
fig.update_layout(xaxis_tickangle=45,
                 width=800,
                 height=600)

# Menampilkan grafik
fig.show()

# Number of Active versus Retired Rockets

How many rockets are active compared to those that are decomissioned? 

In [ ]:
active_retire=df_data['Status'].value_counts()
active_retire

In [ ]:
fig = px.pie(
    values=active_retire.values,
    names=active_retire.index,
    title="Rocket Status Distribution",
    color_discrete_sequence=px.colors.qualitative.Pastel
)

# Menampilkan grafik
fig.show()

# Distribution of Mission Status

How many missions were successful?
How many missions failed?

In [ ]:
mission_status=df_data['Mission_status'].value_counts()
mission_status

In [ ]:
fig = px.bar(
    y=mission_status.values,
    x=mission_status.index,
    title="Mission Status Distribution",
    color_discrete_sequence=px.colors.qualitative.Pastel,
    color=mission_status.index
)
fig.update_layout(showlegend=False)
# Menampilkan grafik
fig.show()

# How Expensive are the Launches? 

Create a histogram and visualise the distribution. The price column is given in USD millions (careful of missing values). 

In [ ]:
# Membuat histogram dengan kurva KDE
plt.figure(figsize=(8, 6))
sns.histplot(df_data["Price"], kde=True, bins=30, color="red")

# Menambahkan label dan judul
plt.title("Distribution of Price", fontsize=16)
plt.xlabel("Price Range", fontsize=12)
plt.ylabel("Frequency", fontsize=12)

# Menampilkan grafik
plt.show()

# Use a Choropleth Map to Show the Number of Launches by Country

* Create a choropleth map using [the plotly documentation](https://plotly.com/python/choropleth-maps/)
* Experiment with [plotly's available colours](https://plotly.com/python/builtin-colorscales/). I quite like the sequential colour `matter` on this map. 
* You'll need to extract a `country` feature as well as change the country names that no longer exist.

Wrangle the Country Names

You'll need to use a 3 letter country code for each country. You might have to change some country names.

* Russia is the Russian Federation
* New Mexico should be USA
* Yellow Sea refers to China
* Shahrud Missile Test Site should be Iran
* Pacific Missile Range Facility should be USA
* Barents Sea should be Russian Federation
* Gran Canaria should be USA


You can use the iso3166 package to convert the country names to Alpha3 format.

In [ ]:
df_data['Location'].unique()

In [ ]:
# Mengambil hanya nama negara dari kolom "Location"
df_data["Country"] = df_data["Location"].str.split(', ').str[-1]

# Menampilkan hasil
df_data[["Location", "Country"]]

In [ ]:
df_data['Country'].unique()

In [ ]:
# Menangani nama-nama negara yang perlu penyesuaian
name_corrections = {
    "Russia": "Russian Federation",
    "New Mexico": "USA",
    "Yellow Sea": "China",
    "Shahrud Missile Test Site": "Iran",
    "Pacific Missile Range Facility": "USA",
    "Barents Sea": "Russian Federation",
    "Gran Canaria": "USA"
}

df_data["Country"] = df_data["Country"].replace(name_corrections)

In [ ]:
country_launches=df_data['Country'].value_counts()
country_launches

In [ ]:
fig = px.choropleth(locations=country_launches.index,
                    color=country_launches.values,
                    locationmode='country names',
                    hover_name=country_launches.index, # column to add to hover information
                    color_continuous_scale=px.colors.sequential.matter,
                   title='Number of Launches by Country')
fig.show()

# Use a Choropleth Map to Show the Number of Failures by Country


In [ ]:
country_launches_status=df_data[['Country', 'Mission_status']].groupby(['Country', 'Mission_status']).value_counts().reset_index()
country_launches_status=country_launches_status.sort_values(by='count', ascending=False)
country_launches_status

In [ ]:
bar = px.bar(country_launches_status, 
               x = 'Country',
               y = 'count',
               title='Country Launches Status', height=800, width=800,
               color='Mission_status')
 
bar.update_layout(xaxis_title='COuntry', yaxis_title='Count')
bar.update_yaxes(categoryorder='total ascending')
bar.show()

In [ ]:
#failure launches by country
failure_data = df_data[df_data["Mission_status"] == "Failure"]
failure_launch_country = failure_data.groupby("Country").size().reset_index(name="Count")
failure_launch_country

In [ ]:
fig = px.choropleth(failure_launch_country,
                    locations='Country',
                    color='Count',
                    locationmode='country names',
                    hover_name='Country', # column to add to hover information
                    color_continuous_scale=px.colors.sequential.matter,
                   title='Number of Failure Launches by Country')
fig.show()

# Create a Plotly Sunburst Chart of the countries, organisations, and mission status. 

In [ ]:
country_organisation_status=df_data.groupby(by=['Country', 'Organisation', 'Mission_status']).count()['Datetime'].reset_index()
country_organisation_status

In [ ]:
fig=px.sunburst(country_organisation_status, path=['Country','Organisation', 'Mission_status'],
               values='Datetime')
fig.show()

# Analyse the Total Amount of Money Spent by Organisation on Space Missions

In [ ]:
grouped_data = df_data[['Organisation', 'Price']].groupby(by='Organisation').sum()
filtered_data = grouped_data[grouped_data['Price'] != 0].sort_values(by='Price', ascending=False)[:20].reset_index()
filtered_data

In [ ]:
bar = px.bar(filtered_data, 
            x = 'Price',
            y = 'Organisation',
             orientation='h',
            title = 'Total Amount of Money Spent by Organisation')
 
bar.update_layout(xaxis_title='Count (*10⁶ $)', yaxis_title='Organisation')
bar.update_yaxes(categoryorder='total ascending')
bar.show()

# Analyse the Amount of Money Spent by Organisation per Launch

In [ ]:
grouped_data = df_data[['Organisation', 'Price']].groupby(by='Organisation').mean()
grouped_data = grouped_data.sort_values(by='Price', ascending=False)[:20].reset_index()
grouped_data

In [ ]:
bar = px.bar(grouped_data, 
            x = 'Price',
            y = 'Organisation',
             orientation='h',
            title = 'Amount of Money Spent by Organisation per Launch')
 
bar.update_layout(xaxis_title='Mean (*10⁶ $)', yaxis_title='Organisation')
bar.update_yaxes(categoryorder='total ascending')
bar.show()

# Chart the Number of Launches per Year

In [ ]:
launches_per_year = df_data.groupby("Year").size().reset_index(name="Count")

filtered_data_year = launches_per_year[launches_per_year["Year"] != 2023]
filtered_data_year

In [ ]:
fig = px.line(
    filtered_data_year,
    x="Year", 
    y="Count",  
    title="Number of Launches per Year",
    labels={"Year": "Year", "Count": "Count"},
    markers=True  
)

# Menyesuaikan tata letak
fig.update_layout(
    xaxis_title="Year",
    yaxis_title="Number of Launches",
    xaxis=dict(dtick=10)  
)

fig.show()

# Chart the Number of Launches Month-on-Month until the Present

Which month has seen the highest number of launches in all time? Superimpose a rolling average on the month on month time series chart. 

In [ ]:
#grouping month
df_data["Month"] = df_data["Datetime"].dt.to_period("M") 
launches_per_month = df_data.groupby("Month").size().reset_index(name="Launch Count")

# convert month to timestamp
launches_per_month["Month"] = launches_per_month["Month"].dt.to_timestamp()

#rolling average
launches_per_month["20_Month_MA"] = launches_per_month["Launch Count"].rolling(window=20).mean()


fig = px.line(
    launches_per_month,
    x="Month",
    y=["Launch Count", "20_Month_MA"],
    title="Number of Launches Month-on-Month with 3-Month Rolling Average",
    labels={"value": "Number of Launches", "variable": "Metric"}
)


fig.update_layout(
    xaxis_title="Month",
    yaxis_title="Number of Launches",
    legend_title="Metrics",
    xaxis=dict(
        tickformat="%b %Y",  
        showgrid=True 
    )
)


fig.show()

# Launches per Month: Which months are most popular and least popular for launches?

Some months have better weather than others. Which time of year seems to be best for space missions?

In [ ]:

df_data["Datetime"] = pd.to_datetime(df_data["Datetime"])


df_data["Month Only"] = df_data["Datetime"].dt.month


print(df_data[["Datetime", "Month Only"]])

In [ ]:
df_data['Month Only'].value_counts().sort_values(ascending=False)

# How has the Launch Price varied Over Time? 

Create a line chart that shows the average price of rocket launches over time. 

In [ ]:
meanprice_per_date=df_data[['Year', 'Price']].groupby(by='Year').mean().reset_index()[:-1]
meanprice_per_date

In [ ]:

plt.figure(figsize=(12, 6))


plt.plot(meanprice_per_date['Year'], meanprice_per_date['Price'], linestyle='-', color='blue')


plt.xlabel('Datetime')
plt.ylabel('Price')
plt.title('Price per Date')  
plt.grid(True)
plt.tight_layout()


plt.show()

# Chart the Number of Launches over Time by the Top 10 Organisations. 

How has the dominance of launches changed over time between the different players? 

In [ ]:
top_10_organisation=df_data['Organisation'].value_counts().sort_values(ascending=False)[:10].index
launches_per_year=df_data[['Year', 'Organisation']].groupby(by=['Year', 'Organisation']).size().reset_index(name='Launch Count')
launches_per_year

In [ ]:
launches_per_year = launches_per_year[launches_per_year['Organisation'].isin(top_10_organisation)]
launches_per_year['Cumulative Launch'] = launches_per_year.groupby(by='Organisation')['Launch Count'].cumsum()
launches_per_year

In [ ]:
l_chart = px.line(launches_per_year,
                  x='Year', 
                  y='Cumulative Launch',
                  color='Organisation',
                  hover_name='Organisation')
 
l_chart.update_layout(xaxis_title='Year',
                      yaxis_title='Cumulative Launch')
 
l_chart.show()

# Cold War Space Race: USA vs USSR

The cold war lasted from the start of the dataset up until 1991. 

In [ ]:
USSR_count = len(df_data[(df_data['Organisation']=='RVSN USSR') & (df_data['Year'] <= 1991)])
USA_count = len(df_data[(df_data['Country']=='USA') & (df_data['Year'] <= 1991)])

data = {
    "Country/Organisation": ["USA", "USSR"],
    "Launch Count": [USA_count, USSR_count]
}
df_pie = pd.DataFrame(data)

fig = px.pie(
    df_pie,
    names="Country/Organisation",  
    values="Launch Count",        
    title="Launch Count Comparison (USA vs USSR) until 1991"
)

fig.show()

## Create a Plotly Pie Chart comparing the total number of launches of the USSR and the USA
Hint: Remember to include former Soviet Republics like Kazakhstan when analysing the total number of launches.

In [ ]:
USSR = df_data[
    ((df_data['Organisation'] == 'RVSN USSR') | 
     (df_data['Country'] == 'Russian Federation') | 
     (df_data['Country'] == 'Kazakhstan')) & 
    (df_data['Year'] <= 1991)
]

USA = df_data[(df_data['Country'] == 'USA') & (df_data['Year'] <= 1991)]

USSR_count = len(USSR)
USA_count = len(USA)

data = {
    "Country/Organisation": ["USA", "USSR"],
    "Launch Count": [USA_count, USSR_count]
}
df_pie = pd.DataFrame(data)

fig = px.pie(
    df_pie,
    names="Country/Organisation",  
    values="Launch Count",        
    title="Total Launch Count Comparison (USA vs USSR)"
)

fig.show()

## Create a Chart that Shows the Total Number of Launches Year-On-Year by the Two Superpowers

In [ ]:
# Number of launches year-on-year for USSR and USA
USSR_launches = USSR.groupby('Year').size().reset_index(name='Launch Count')
USA_launches = USA.groupby('Year').size().reset_index(name='Launch Count')

USSR_launches['Country'] = 'USSR'
USA_launches['Country'] = 'USA'

combined_launches = pd.concat([USSR_launches, USA_launches])
combined_launches

In [ ]:
fig = px.line(
    combined_launches,
    x='Year',
    y='Launch Count',
    color='Country',
    title='Year-on-Year Total Rocket Launches by USSR and USA',
    labels={
        'Year': 'Year',
        'Launch Count': 'Number of Launches',
        'Country': 'Country'
    },
    markers=True
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Launches',
    width=1000,
    height=600,
    xaxis=dict(
        dtick=1 
    )
)

fig.show()

## Chart the Total Number of Mission Failures Year on Year.

In [ ]:
USSR_failure_count = USSR[USSR['Mission_status']=='Failure'].groupby(by='Year').size().reset_index(name='Failure Count')
USA_failure_count = USA[USA['Mission_status']=='Failure'].groupby(by='Year').size().reset_index(name='Failure Count')

USSR_failure_count['Country'] = 'USSR'
USA_failure_count['Country'] = 'USA'

combined_failure_launches = pd.concat([USSR_failure_count, USA_failure_count])
combined_failure_launches

In [ ]:
fig = px.line(
    combined_failure_launches,
    x='Year',
    y='Failure Count',
    color='Country',
    title='Year-on-Year Total Rocket Failure by USSR and USA',
    labels={
        'Year': 'Year',
        'Failure Count': 'Number of Failure',
        'Country': 'Country'
    },
    markers=True
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Failure',
    width=1000,
    height=600,
    xaxis=dict(
        dtick=1 
    )
)

fig.show()

## Chart the Percentage of Failures over Time

Did failures go up or down over time? Did the countries get better at minimising risk and improving their chances of success over time? 

In [ ]:
failure_percent_USA = pd.merge(USA_failure_count, USA_launches, on='Year')
failure_percent_USA['Failure Percentage'] = (failure_percent_USA['Failure Count']/failure_percent_USA['Launch Count'])*100
failure_percent_USA

In [ ]:
failure_percent_USSR = pd.merge(USSR_failure_count, USSR_launches, on='Year')
failure_percent_USSR['Failure Percentage'] = (failure_percent_USSR['Failure Count']/failure_percent_USSR['Launch Count'])*100
failure_percent_USSR

In [ ]:
combined_failure_percent = pd.concat([failure_percent_USSR, failure_percent_USA])
combined_failure_percent

In [ ]:
fig = px.line(
    combined_failure_percent,
    x='Year',
    y='Failure Percentage',
    color='Country_x',
    title='Year-on-Year Percentage Rocket Failure by USSR and USA',
    labels={
        'Year': 'Year',
        'Failure Percentage': 'Failure Percentage',
        'Country': 'Country'
    },
    markers=True
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Failure Percentage',
    width=1000,
    height=600,
    xaxis=dict(
        dtick=1 
    )
)

fig.show()

# For Every Year Show which Country was in the Lead in terms of Total Number of Launches up to and including 2022)

Do the results change if we only look at the number of successful launches? 

In [ ]:
launches_per_country_year = df_data.groupby(['Year', 'Country']).size().reset_index(name='Launch Count')


leader_per_year = launches_per_country_year.loc[
    launches_per_country_year.groupby('Year')['Launch Count'].idxmax()
][:-1].reset_index(drop=True)


leader_per_year

In [ ]:
fig = px.bar(
    leader_per_year,
    y="Launch Count",
    x="Year",
    color="Country",
    title="Yearly Launch Leaders by Country"
)
fig.show()

# Create a Year-on-Year Chart Showing the Organisation Doing the Most Number of Launches

Which organisation was dominant in the 1970s and 1980s? Which organisation was dominant in 2018, 2019 and 2020? 

In [ ]:
launches_per_org_year = df_data.groupby(['Year', 'Organisation']).size().reset_index(name='Launch Count')

# Menemukan organisasi dengan jumlah peluncuran terbanyak untuk setiap tahun
top_org_per_year = launches_per_org_year.loc[
    launches_per_org_year.groupby('Year')['Launch Count'].idxmax()
].reset_index(drop=True)

fig = px.bar(
    top_org_per_year,
    x='Year',
    y='Launch Count',
    color='Organisation',
    title='Year-on-Year Chart of Organisation with Most Launches',
    labels={
        'Year': 'Year',
        'Launch Count': 'Number of Launches',
        'Organisation': 'Organisation'
    }
)

fig.update_layout(
    xaxis_title='Year',
    yaxis_title='Number of Launches' 
)

fig.show()